In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # visualização gráfica
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.tree import plot_tree
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def espec_sens(observado,predicts):
    
    # adicionar objeto com os valores dos predicts
    # values = predicts.values
    values = predicts
    
    # range dos cutoffs a serem analisados em steps de 0.01
    cutoffs = np.arange(0,1.01,0.01)
    
    # Listas que receberão os resultados de especificidade e sensitividade
    lista_sensitividade = []
    lista_especificidade = []
    
    for cutoff in cutoffs:
        
        predicao_binaria = []
        
        # Definindo resultado binário de acordo com o predict
        for item in values:
            if item >= cutoff:
                predicao_binaria.append(1)
            else:
                predicao_binaria.append(0)
                
        # Cálculo da sensitividade e especificidade no cutoff
        sensitividade = recall_score(observado, predicao_binaria, pos_label=1)
        especificidade = recall_score(observado, predicao_binaria, pos_label=0)
        
        # Adicionar valores nas listas
        lista_sensitividade.append(sensitividade)
        lista_especificidade.append(especificidade)
        
    # Criar dataframe com os resultados nos seus respectivos cutoffs
    resultado = pd.DataFrame({'cutoffs':cutoffs,'sensitividade':lista_sensitividade,'especificidade':lista_especificidade})
    return resultado

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df_base_nativa = pd.read_excel('DadosClientesProjeto.xlsx')
df_base_nativa.info()

In [ ]:
# replace de valores em brancos ou espaços para NaN
df_base_nativa.replace(r'^\s*$', np.nan, regex=True, inplace=True)
# confere valores NaN
df_base_nativa.isna().sum()

In [ ]:
df_base = df_base_nativa.dropna()
df_base.info()

In [ ]:
df_base.duplicated().sum()

In [ ]:
lista_colunas_qualitativas = ['Segmento', 'UF', 'Municipio', 'ContratoAtivo', 'FretePadrao', 'DescCondPag']

In [ ]:
lista_colunas_quantitativas = ['DiasDesdePrimeiroFaturamento', 'DiasDesdeUltimoFaturamento', 'MediaMargemRelativa',
                                'SomaValorFaturado', 'ProdutosDiferentesComprados', 'TicketMedio', 'QuantidadePedidos',
                                'QuantidadeMesesComprando', 'TaxaConversaoOrcamentos', 'MesesConsecutivosComprando',
                                'MediaFrete']

In [ ]:
df_base.groupby('Churn').describe().T

In [ ]:
df_base.describe().T

In [ ]:
print(df_base.describe().T.to_string())

In [ ]:
for coluna in lista_colunas_qualitativas:
    print(df_base[coluna].value_counts())
    print('-=' * 20)

In [ ]:
total_clientes = len(df_base)

for col in lista_colunas_qualitativas:
    # conta o número de clientes por categoria e divide pelo número total de clientes
    # ou seja, porcentagem de clientes por categoria
    temp_df = pd.Series(df_base[col].value_counts() / total_clientes)

    # faz um gráfico com as porcetanges obtidas
    fig = temp_df.sort_values(ascending=False).plot.bar()
    fig.set_xlabel(col)

    # adiciona linha em 1%
    fig.axhline(y=0.01, color='red')
    fig.set_ylabel('Porcentagem de clientes')
    plt.show()

In [ ]:
qual_muitas_categ = ['Municipio', 'DescCondPag']

In [ ]:
def agrupa_categorias_raras(df, var, cutoff):
    total_clientes = len(df)
    # calcula a porcentagem de clientes em cada categoria
    temp_df = pd.Series(df[var].value_counts() / total_clientes)

    # cria um dicionário para substituir categorias raras com string 'Outras'
    grouping_dict = {
        k: ('Outros' if k not in temp_df[temp_df >= cutoff].index else k)
        for k in temp_df.index
    }

    # substitui categorias raras
    tmp = df[var].map(grouping_dict)

    return tmp

In [ ]:
for col in lista_colunas_qualitativas:
    if col not in qual_muitas_categ:
        df_base[f'{col}_agrupado'] = agrupa_categorias_raras(df_base, f'{col}', 0.01)
    elif col in qual_muitas_categ:
        df_base[f'{col}_agrupado'] = agrupa_categorias_raras(df_base, f'{col}', 0.005)

In [ ]:
for coluna in lista_colunas_qualitativas:
    print(df_base[f'{coluna}_agrupado'].value_counts())
    print('-=' * 20)
    print(df_base[f'{coluna}_agrupado'].value_counts(normalize=True).mul(100))
    print('-=' * 20)

In [ ]:
for col in lista_colunas_qualitativas:
    print(df_base.groupby(['Churn', f'{col}_agrupado'])['Churn'].count())
    print('-=' * 20)
    print(df_base.groupby(['Churn', f'{col}_agrupado'])['Churn'].count().div(total_clientes).mul(100))
    print('-=' * 20)

In [ ]:
print(df_base.groupby(['Churn'])['Churn'].count())
print(df_base.groupby(['Churn'])['Churn'].count().div(total_clientes).mul(100))
print(len(df_base['Churn']))

In [ ]:
for col in lista_colunas_qualitativas:
    print('Não')
    print(df_base[df_base['Churn'] == 0][f'{col}_agrupado'].value_counts(normalize=True).mul(100))
    print('-=' * 20)
    print('Sim')
    print(df_base[df_base['Churn'] == 1][f'{col}_agrupado'].value_counts(normalize=True).mul(100))
    print('-=' * 20)
    print(df_base.groupby(['Churn', f'{col}_agrupado'])['Churn'].count())
    print('-=' * 20)

In [ ]:
nova_lista_colunas_qualitativas = ['Segmento_agrupado', 'UF_agrupado', 'Municipio_agrupado', 'ContratoAtivo_agrupado', 
                                   'FretePadrao_agrupado', 'DescCondPag_agrupado']
df_base_tratada = df_base.drop(lista_colunas_qualitativas, axis=1)
df_base_tratada

In [ ]:
df_base_dummies = pd.get_dummies(df_base_tratada,
                                 columns=nova_lista_colunas_qualitativas,
                                 dtype=int,
                                 drop_first=True)
df_base_dummies

In [ ]:
X = df_base_dummies.drop(columns=['Churn'])
y = df_base_dummies['Churn']

# Vamos escolher 70% das observações para treino e 30% para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=100)

In [ ]:
y_train.info()

In [ ]:
X_train.info()

In [ ]:
y_test.info()

In [ ]:
X_test.info()

In [ ]:
reg_log = LogisticRegression(random_state=100)
np.random.seed(100)

# Defining hyperparameters to search over
params = {
    'penalty':['l1','l2','elasticnet','none'],
    'C' : [0.001, 0.01, 0.1, 1, 10, 100],
    'fit_intercept': [True, False],
    'solver': ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter'  : [100, 1000, 2500, 5000, 10000]
}

# Defining randomized search
reg_log_cv = RandomizedSearchCV(estimator=reg_log, 
                            param_distributions=params,
                            random_state=100,
                            n_jobs=-1)

# Fitting randomized search to training data
reg_log_cv.fit(X_train, y_train)
print(reg_log_cv.best_params_)

# Model predictions 
y_pred_reg_log = reg_log_cv.predict(X_test)
prob_pos = reg_log_cv.predict_proba(X_test)[:,1]

# Model performance
reg_log_accuracy = accuracy_score(y_test, y_pred_reg_log)
print(("Acurácia: {:.4f}".format(reg_log_accuracy)))

reg_log_precision = precision_score(y_test, y_pred_reg_log)
print("Precisão: {:.4f}".format(reg_log_precision))

reg_log_recall = recall_score(y_test, y_pred_reg_log)
print("Sensitividade: {:.4f}".format(reg_log_recall))

reg_log_fpr = recall_score(y_test, y_pred_reg_log, pos_label=0)
print("Especificidade: {:.4f}".format(reg_log_fpr))

reg_log_f1 = f1_score(y_test, y_pred_reg_log)
print("Pontuação F1: {:.4f}".format(reg_log_f1))

reg_log_auc_value = roc_auc_score(y_test, prob_pos)
print("AUC: {:.4f}".format(reg_log_auc_value))

In [ ]:
cm = confusion_matrix(y_pred_reg_log, y_test)
cm_reg_log = ConfusionMatrixDisplay(cm)

plt.rcParams['figure.dpi'] = 100
cm_reg_log.plot(colorbar=False, cmap='viridis')
plt.title('Regressão logística: Teste')
plt.xlabel('Observado (Real)')
plt.ylabel('Classificado (Modelo)')
plt.show()

In [ ]:
# Até o momento, foram extraídos 3 vetores: 'sensitividade',
#'especificidade' e 'cutoffs'. Assim, criamos um dataframe que contém
#os vetores mencionados

dados_plotagem = espec_sens(observado = y_test,
                            predicts = y_pred_reg_log)
dados_plotagem

# Plotagem de um gráfico que mostra a variação da especificidade e da
#sensitividade em função do cutoff

plt.figure(figsize=(15,10), dpi=50)
with plt.style.context('seaborn-v0_8-whitegrid'):
    plt.plot(dados_plotagem.cutoffs,dados_plotagem.sensitividade, marker='o',
         color='indigo', markersize=8)
    plt.plot(dados_plotagem.cutoffs,dados_plotagem.especificidade, marker='o',
         color='limegreen', markersize=8)
plt.xlabel('Cuttoff', fontsize=20)
plt.ylabel('Sensitividade / Especificidade', fontsize=20)
plt.xticks(np.arange(0, 1.1, 0.2), fontsize=14)
plt.yticks(np.arange(0, 1.1, 0.2), fontsize=14)
plt.legend(['Sensitividade', 'Especificidade'], fontsize=20)
plt.show()

In [ ]:
# Parametrizando a função da curva ROC (real vs. previsto)
fpr_reg_log, tpr_reg_log, thresholds_reg_log = roc_curve(y_test, prob_pos)
roc_auc_xgb = auc(fpr_reg_log, tpr_reg_log)

# Plotando a curva ROC
plt.figure(figsize=(15,10), dpi=50)
plt.plot(fpr_reg_log, tpr_reg_log, color='green', linewidth=4)
plt.plot(fpr_reg_log, fpr_reg_log, color='gray', linestyle='dashed')
plt.title('AUC-ROC regressão logística: %g' % round(roc_auc_xgb, 3), fontsize=22)
plt.xlabel('1 - Especificidade', fontsize=20)
plt.ylabel('Sensibilidade', fontsize=20)
plt.xticks(np.arange(0, 1.1, 0.2), fontsize=14)
plt.yticks(np.arange(0, 1.1, 0.2), fontsize=14)
plt.show()

In [ ]:
dt = DecisionTreeClassifier(random_state=100)
np.random.seed(100)

params = {
    'criterion': ['gini', 'entropy', 'logloss'],
    'splitter': ['best', 'random'],
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, None],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': ['auto', 'sqrt', 'log2', None]
}

cv_dt = RandomizedSearchCV(
    dt, 
    param_distributions=params, 
    random_state=100,
    n_jobs=-1
)

cv_dt.fit(X_train, y_train)
print(cv_dt.best_params_)

best_dt = cv_dt.best_estimator_
best_dt.fit(X_train, y_train)

y_pred_dt = best_dt.predict(X_test)
prob_pos = best_dt.predict_proba(X_test)[:,1]

dt_accuracy = accuracy_score(y_test, y_pred_dt)
print(("Acurácia: {:.4f}".format(dt_accuracy)))

dt_precision = precision_score(y_test, y_pred_dt)
print("Precisão: {:.4f}".format(dt_precision))

dt_recall = recall_score(y_test, y_pred_dt)
print("Sensitividade: {:.4f}".format(dt_recall))

dt_fpr = recall_score(y_test, y_pred_dt, pos_label=0)
print("Especificidade: {:.4f}".format(dt_fpr))

dt_f1 = f1_score(y_test, y_pred_dt)
print("Pontuação F1: {:.4f}".format(dt_f1))

dt_auc_value = roc_auc_score(y_test, prob_pos)
print("AUC: {:.4f}".format(dt_auc_value))

In [ ]:
plt.figure(figsize=(20,10), dpi=100)
plot_tree(best_dt,
          feature_names=X.columns.tolist(),
          class_names=['Não churner','Churner'],
          proportion=False,
          filled=True,
          node_ids=True)
plt.show()

In [ ]:
#%% Matriz de confusão (base de teste)
cm = confusion_matrix(y_pred_dt, y_test)
cm_dt = ConfusionMatrixDisplay(cm)

plt.rcParams['figure.dpi'] = 100
cm_dt.plot(colorbar=False, cmap='viridis')
plt.title('Árvore de Decisão: Teste')
plt.xlabel('Observado (Real)')
plt.ylabel('Classificado (Modelo)')
plt.show()

In [ ]:
rf = RandomForestClassifier(random_state=100)
np.random.seed(100)

params = {
    'n_estimators': [10, 20, 50, 100, 200, 400],
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 30, None],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': ['auto', 'sqrt', 'log2', None],
    'bootstrap': [True, False],
    'oob_score': [True, False],
}

cv_rf = RandomizedSearchCV(
    rf,
    param_distributions=params,
    random_state=100,
    n_jobs=-1
)

cv_rf.fit(X_train, y_train)
print(cv_rf.best_params_)

# Model predictions
y_pred_rf = cv_rf.predict(X_test)
prob_pos = cv_rf.predict_proba(X_test)[:,1]

rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(("Acurácia: {:.4f}".format(rf_accuracy)))

rf_precision = precision_score(y_test, y_pred_rf)
print("Precisão: {:.4f}".format(rf_precision))

rf_recall = recall_score(y_test, y_pred_rf)
print("Sensitividade: {:.4f}".format(rf_recall))

rf_fpr = recall_score(y_test, y_pred_rf, pos_label=0)
print("Especificidade: {:.4f}".format(rf_fpr))

rf_f1 = f1_score(y_test, y_pred_rf)
print("Pontuação F1: {:.4f}".format(rf_f1))

rf_auc_value = roc_auc_score(y_test, prob_pos)
print("AUC: {:.4f}".format(rf_auc_value))

In [ ]:
cm = confusion_matrix(y_pred_rf, y_test)
cm_rf = ConfusionMatrixDisplay(cm)

plt.rcParams['figure.dpi'] = 100
cm_rf.plot(colorbar=False, cmap='viridis')
plt.title('Random forest: Teste')
plt.xlabel('Observado (Real)')
plt.ylabel('Classificado (Modelo)')
plt.show()

In [ ]:
xgb = XGBClassifier(random_state=100)
np.random.seed(100)

params = {
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'n_estimators': [50, 100, 200, 300, 400, 500],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'min_child_weight': [1, 3, 5, 7, 9],
    'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'objective': ['reg:squarederror', 'reg:squaredlogerror', 'reg:logistic', 
                  'binary:logistic', 'binary:logitraw', 'binary:hinge',
                  'multi:softmax', 'multi:softprob'],
    'eval_metric': ['rmse', 'mae', 'logloss', 'error', 'merro',
                    'mlogloss', 'auc', 'aucpr'],
}

cv_xgb = RandomizedSearchCV(
    xgb,
    param_distributions=params,
    random_state=100,
    n_jobs=-1
)

cv_xgb.fit(X_train, y_train)
print(cv_xgb.best_params_)

# Mode predictions 
y_pred_xgb = cv_xgb.predict(X_test)
prob_pos = cv_xgb.predict_proba(X_test)[:,1]

xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
print(("Acurácia: {:.4f}".format(xgb_accuracy)))

xgb_precision = precision_score(y_test, y_pred_xgb)
print("Precisão: {:.4f}".format(xgb_precision))

xgb_recall = recall_score(y_test, y_pred_xgb)
print("Sensitividade: {:.4f}".format(xgb_recall))

xgb_fpr = recall_score(y_test, y_pred_xgb, pos_label=0)
print("Especificidade: {:.4f}".format(xgb_fpr))

xgb_f1 = f1_score(y_test, y_pred_xgb)
print("Pontuação F1: {:.4f}".format(xgb_f1))

xgb_auc_value = roc_auc_score(y_test, prob_pos)
print("AUC: {:.4f}".format(xgb_auc_value))

In [ ]:
cm = confusion_matrix(y_pred_xgb, y_test)
cm_xgb = ConfusionMatrixDisplay(cm)

plt.rcParams['figure.dpi'] = 100
cm_xgb.plot(colorbar=False, cmap='viridis')
plt.title('Random forest: Teste')
plt.xlabel('Observado (Real)')
plt.ylabel('Classificado (Modelo)')
plt.show()

In [ ]:
# Gere suas predições separadas para cada modelo
# Exemplo (substitua pelos seus próprios):
# y_pred_log, y_pred_dt, y_pred_rf, y_pred_xgb = ...

fig, axes = plt.subplots(2, 2, figsize=(10, 8))  # Cria grid 2x2
plt.subplots_adjust(hspace=0.4, wspace=0.4)  # Espaçamento entre os subplots

# 1. Regressão Logística
cm = confusion_matrix(y_test, y_pred_reg_log)
ConfusionMatrixDisplay(cm).plot(ax=axes[0, 0], colorbar=False, cmap='viridis')
axes[0, 0].set_title('Regressão logística: Teste')
axes[0, 0].set_xlabel('Observado (Real)')
axes[0, 0].set_ylabel('Classificado (Modelo)')

# 2. Árvore de Decisão
cm = confusion_matrix(y_test, y_pred_dt)
ConfusionMatrixDisplay(cm).plot(ax=axes[0, 1], colorbar=False, cmap='viridis')
axes[0, 1].set_title('Árvore de Decisão: Teste')
axes[0, 1].set_xlabel('Observado (Real)')
axes[0, 1].set_ylabel('Classificado (Modelo)')

# 3. Random Forest
cm = confusion_matrix(y_test, y_pred_rf)
ConfusionMatrixDisplay(cm).plot(ax=axes[1, 0], colorbar=False, cmap='viridis')
axes[1, 0].set_title('Random Forest: Teste')
axes[1, 0].set_xlabel('Observado (Real)')
axes[1, 0].set_ylabel('Classificado (Modelo)')

# 4. XGBoost
cm = confusion_matrix(y_test, y_pred_xgb)
ConfusionMatrixDisplay(cm).plot(ax=axes[1, 1], colorbar=False, cmap='viridis')
axes[1, 1].set_title('XGBoost: Teste')
axes[1, 1].set_xlabel('Observado (Real)')
axes[1, 1].set_ylabel('Classificado (Modelo)')

plt.tight_layout()
plt.show()